# Introduction to `scikit-learn`

In this notebook we will introduce the `scikit-learn` package for fitting linear models. We will also discuss the advantages of using `scikit-learn` over implementing models yourself using `numpy`.

In [1]:
import pandas as pd
import numpy as np
from seaborn import load_dataset

In [2]:
data = load_dataset("mpg")

# Drop rows with null values for this example
data = data[~data.isna().any(axis=1)]

# Focus on only quantitative terms for this example
data = data.drop(columns=['origin', 'name'])
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## Fitting linear models using the normal equations

Let's remind ourselves of how we have been fitting models so far.

In [3]:
# Create a column of 1's to represent the intercept term
data['intercept'] = pd.Series(data=[1] * len(data), index=data.index)

# Create the design matrix X and the target vector y
y = data['mpg'].to_numpy()
X = data.drop(columns=['mpg']).to_numpy()

# Fit the model (i.e. set up and solve the normal equations)
theta_hat = np.linalg.inv(X.T @ X) @ (X.T @ y)
theta_hat

array([-3.29859089e-01,  7.67843024e-03, -3.91355574e-04, -6.79461791e-03,
        8.52732469e-02,  7.53367180e-01, -1.45352505e+01])

To make predictions using our model, we use the parameter vector $\hat{\theta}$ as shown below. This effectively means our entire model is captured in the $\hat{\theta}$ vector.

In [4]:
# Make predictions using the model
preds = X @ theta_hat
preds[:10]

array([15.08291904, 14.07257469, 15.53631544, 15.53447451, 15.28640745,
       10.13543367, 10.14518132, 10.2823774 ,  9.75375835, 13.04735326])

## Fitting linear models using `scikit-learn`

In [5]:
# Load a fresh dataset for scikit-learn demo
data = load_dataset("mpg")
data = data[~data.isna().any(axis=1)]
data = data.drop(columns=['origin', 'name'])
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


The `scikit-learn` package has many features that are useful for modeling, feature engineering, and more. In this notebook, we will show the functionality `scikit-learn` has for modeling.

To fit an ordinary least squares model using `scikit-learn`, we first have to import the `LinearRegression` [object](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) from `scikit-learn`:

In [6]:
# Import the LinearRegression object, which represents an ordinary least squares model
from sklearn.linear_model import LinearRegression

The next step is to create an instance of the LinearRegression object:

In [7]:
# Instantiate a model
model = LinearRegression()

Next, we fit our model to the data:

In [10]:
# Create the design matrix X and the target vector y
y = data['mpg']
X = data.drop(columns=['mpg'])

# Fit the model (notice no normal equations!)
model.fit(X, y)

LinearRegression()

Now we can use our model to make predictions:

In [11]:
# Make predictions using the model
model.predict(X)[:10]

array([15.08291904, 14.07257469, 15.53631544, 15.53447451, 15.28640745,
       10.13543367, 10.14518132, 10.2823774 ,  9.75375835, 13.04735326])

What happened to the $\hat{\theta}$ vector? How are we able to make predictions without computing $\hat{\theta}$?

When we fit the model using `model.fit(X, y)`, `scikit-learn` computes and saves $\hat{\theta}$ so that when we do `model.predict(X)`, the model already knows what $\hat{\theta}$ is and can use it to make predictions.

We can even look at the $\hat{\theta}$ vector computed by `model.fit(X, y)`:

In [12]:
model.coef_

array([-3.29859089e-01,  7.67843024e-03, -3.91355574e-04, -6.79461791e-03,
        8.52732469e-02,  7.53367180e-01])

There are only 6 coefficients! What happened to the intercept term?

In [14]:
model.intercept_

-14.535250480506168

By default, `scikit-learn` will add an intercept term to the model for you. This means you don't have to go through the trouble of adding it yourself! This is just one of the many advantages using `scikit-learn` has.

## Advantages of scikit-learn

While this example of an OLS model might not clearly show why `scikit-learn` is preferred over implementing everything yourself using something like `numpy`, there are a few important advantages to using `scikit-learn`:

- using `scikit-learn` abstracts model details (e.g. normal equations)
- consistent interface across models makes it easy to try different models (example below)
- interoperability with other `scikit-learn` features

In [15]:
from sklearn.linear_model import *
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

model_types = [
    LinearRegression,
    Ridge,
    Lasso,
    ElasticNet,
    DecisionTreeRegressor
]

for model_type in model_types:
    model = model_type().fit(X, y)
    preds = model.predict(X)
    print(f"{model} RMSE: {mean_squared_error(y, preds, squared=False)}")

LinearRegression() RMSE: 3.4044340177796406
Ridge() RMSE: 3.404434439176605
Lasso() RMSE: 3.4249669617779075
ElasticNet() RMSE: 3.4214869327992687
DecisionTreeRegressor() RMSE: 0.0
